In [1]:
import os
import cv2
import numpy as np
from cvl.dataset import OnlineTrackingBenchmark
from cvl.trackers import MOSSETracker, DCFMOSSETracker
from cvl.lib import get_roi, resume_performance, get_arguments
import matplotlib.pyplot as plt
from copy import copy, deepcopy
import torch
from tqdm.notebook import tqdm as tqdm
import json

In [2]:
ds_path = "Mini-OTB/"

In [3]:
seq_list = os.listdir(ds_path)

In [4]:
dataset = OnlineTrackingBenchmark(ds_path)

# 1) Choose the tracker

In [5]:
TRACKER = "vgg16" # ["mosse", "resnet", "mobilenet", "alexnet", "vgg16", "hand_f"]

In [6]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

/home/tmr/miniconda3/envs/torch/lib/python3.6/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729128610/work/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [7]:
if TRACKER == "mosse":
    tracker = MOSSETracker()
    squared = False
elif TRACKER in ["resnet", "mobilenet", "alexnet", "vgg16", "hand_f"]:
    tracker = DCFMOSSETracker(dev = device, features = TRACKER)
    squared = True

In [8]:
results = {}

In [9]:
for idx, seq in enumerate(dataset):
    print(f"----------------------------Evaluating video named {seq.sequence_name}----------------------------------")
    bboxes = []
    for frame_idx, frame in enumerate(seq):
        image_color = frame['image']
        if TRACKER == "mosse":
            image = np.sum(image_color, 2) / 3 # grayscale
        else:
            image = np.transpose(np.float64(image_color), (2, 0, 1))
        if frame_idx == 0:
            bbox = copy(frame['bounding_box'])
            aaaa = bbox
            if bbox.width % 2 == 0:
                bbox.width += 1
            if bbox.height % 2 == 0:
                bbox.height += 1
            roi = get_roi(bbox, squared = squared) # get roi slightly bigger that bbox
            tracker.start(image, bbox, roi) # first frame approach
        else:
            tracker.detect(image)
            tracker.update()
        bboxes.append(copy(tracker.bbox))
    ious = dataset.calculate_per_frame_iou(idx, bboxes)
    results[idx] = ious

----------------------------Evaluating video named Basketball----------------------------------
----------------------------Evaluating video named Biker----------------------------------
----------------------------Evaluating video named BlurBody----------------------------------
----------------------------Evaluating video named BlurCar3----------------------------------
----------------------------Evaluating video named Bolt----------------------------------
----------------------------Evaluating video named Box----------------------------------
----------------------------Evaluating video named CarScale----------------------------------
----------------------------Evaluating video named Coke----------------------------------
----------------------------Evaluating video named Coupon----------------------------------
----------------------------Evaluating video named Crossing----------------------------------
----------------------------Evaluating video named Crowds-------------------

In [10]:
result_file = f"../results/avg_precision/best_alexnet.json"

In [11]:
with open(result_file, 'w') as fp:
    json.dump(results, fp)

In [12]:
def get_avg_precision(list_ious, threshold = 0.5):
    return np.sum((np.array(list_ious)>threshold)) / len(list_ious)

In [13]:
ious = []

In [14]:
for iou in list(results.values()):
    ious.extend(iou)

In [15]:
get_avg_precision(ious, threshold=0.5)

0.46111334405144694